In [1]:
import requests
from shapely.geometry import LineString
import pandas as pd
import json
import osmnx as ox
import pyrosm
import folium
import numpy as np

ox.__version__

useful_tags_path = ['bridge', 'tunnel', 'oneway', 'lanes', 'ref', 'name',
                    'highway', 'maxspeed', 'service', 'access', 'area',
                    'landuse', 'width', 'est_width', 'junction', 'traffic_signals']
ox.utils.config(useful_tags_path)

In [2]:
ADDRESS = "Mayfair, London"
G = ox.graph_from_address(ADDRESS, network_type="drive", dist=300)

/Users/khalid-alsheeb/opt/anaconda3/envs/kurf/lib/python3.10/site-packages/osmnx/utils_geo.py:280: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in geometry:
/Users/khalid-alsheeb/opt/anaconda3/envs/kurf/lib/python3.10/site-packages/osmnx/utils_geo.py:374: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for poly in multipoly:
/Users/khalid-alsheeb/opt/anaconda3/envs/kurf/lib/python3.10/site-packages/osmnx/utils_geo.py:374: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for poly in multipoly:


In [3]:
nodes, edges = ox.graph_to_gdfs(G)
#print(edges.name.value_counts())
#print(edges.name.explode().unique())

# edges = init_traffic_column(edges)
# edges.columns


In [4]:
def init_traffic_columns(edges):
    if not "current_speed" in edges:
        edges["current_speed"] = np.nan
    return edges

def get_full_traffic_data(edges):
    
    names = edges.name.value_counts().index
    
    for name in names:
        get_specific_traffic_data(edges, name)
        
    return edges

def get_specific_traffic_data(edges, name):
    # Create request URL
    BASE_URL = "https://api.tomtom.com/"
    API_KEY = "ThfURaOhGRaty7mx9AKALwrbyIZApHd8"
    #zoom = 22
    TRAFFIC_URL = "traffic/services/4/flowSegmentData/absolute/22/json?key="
    UNIT = "unit=kmph"
    ROAD_CLOSURE = "roadClosure=True"
    PARAMS = "&" + UNIT + "&" + ROAD_CLOSURE
    
    coordinates = list(edges.loc[edges['name'] == name].iloc[0].geometry.coords)[0]
    longitude = str(coordinates[0])
    latitude = str(coordinates[1])
    location = "point=" + latitude + ","+ longitude

    request_url = BASE_URL + TRAFFIC_URL + API_KEY + "&" + location + PARAMS
    # Get data
    response = requests.get(request_url)
    data = response.json()['flowSegmentData']
    traffic = pd.json_normalize(data)
    
    edges.loc[edges['name'] == name, 'road_type'] = traffic.frc[0]
    edges.loc[edges['name'] == name, 'current_speed_kmph'] = traffic.currentSpeed[0]
    edges.loc[edges['name'] == name, 'free_flow_speed_kmph'] = traffic.freeFlowSpeed[0]
    edges.loc[edges['name'] == name, 'traffic_confidence'] = traffic.confidence[0]
    edges.loc[edges['name'] == name, 'road_closure'] = traffic.roadClosure[0]

In [5]:
edges.loc[edges['name'] == 'Davies Street']

,,,osmid,lanes,name,highway,maxspeed,length,geometry,oneway,ref,est_width,width,access
u,v,key,,,,,,,,,,,,
21651907,25256612,0,198190853,NaN,Davies Street,residential,20 mph,61.648,"LINESTRING (-0.14781 51.51160, -0.14787 51.511...",no,NaN,NaN,NaN,NaN
25256296,811961032,0,165582921,2,Davies Street,tertiary,20 mph,57.049,"LINESTRING (-0.14685 51.51001, -0.14699 51.510...",no,NaN,NaN,NaN,no
25256330,21651907,0,236034852,NaN,Davies Street,tertiary,20 mph,78.582,"LINESTRING (-0.14749 51.51092, -0.14775 51.511...",no,NaN,NaN,NaN,NaN
25256612,25256623,0,198190853,NaN,Davies Street,residential,20 mph,61.358,"LINESTRING (-0.14806 51.51213, -0.14822 51.512...",no,NaN,NaN,NaN,NaN
25256615,25256623,0,202195797,NaN,Davies Street,residential,20 mph,45.541,"LINESTRING (-0.14846 51.51306, -0.14839 51.512...",no,NaN,NaN,NaN,NaN
25257070,25256330,0,236034852,NaN,Davies Street,tertiary,20 mph,31.843,"LINESTRING (-0.14737 51.51064, -0.14749 51.51092)",no,NaN,NaN,NaN,NaN
811961032,25257070,0,236034852,NaN,Davies Street,tertiary,20 mph,31.669,"LINESTRING (-0.14731 51.51037, -0.14727 51.510...",no,NaN,NaN,NaN,NaN
7049006680,25256615,0,202195797,NaN,Davies Street,residential,20 mph,29.996,"LINESTRING (-0.14858 51.51332, -0.14846 51.51306)",no,NaN,NaN,NaN,NaN


In [6]:
get_specific_traffic_data(edges, 'Davies Street')

In [7]:
edges.loc[edges['name'] == 'Davies Street']

,,,osmid,lanes,name,highway,maxspeed,length,geometry,oneway,ref,est_width,width,access,road_type,current_speed_kmph,free_flow_speed_kmph,traffic_confidence,road_closure
u,v,key,,,,,,,,,,,,,,,,,
21651907,25256612,0,198190853,NaN,Davies Street,residential,20 mph,61.648,"LINESTRING (-0.14781 51.51160, -0.14787 51.511...",no,NaN,NaN,NaN,NaN,FRC5,18.0,18.0,1.0,False
25256296,811961032,0,165582921,2,Davies Street,tertiary,20 mph,57.049,"LINESTRING (-0.14685 51.51001, -0.14699 51.510...",no,NaN,NaN,NaN,no,FRC5,18.0,18.0,1.0,False
25256330,21651907,0,236034852,NaN,Davies Street,tertiary,20 mph,78.582,"LINESTRING (-0.14749 51.51092, -0.14775 51.511...",no,NaN,NaN,NaN,NaN,FRC5,18.0,18.0,1.0,False
25256612,25256623,0,198190853,NaN,Davies Street,residential,20 mph,61.358,"LINESTRING (-0.14806 51.51213, -0.14822 51.512...",no,NaN,NaN,NaN,NaN,FRC5,18.0,18.0,1.0,False
25256615,25256623,0,202195797,NaN,Davies Street,residential,20 mph,45.541,"LINESTRING (-0.14846 51.51306, -0.14839 51.512...",no,NaN,NaN,NaN,NaN,FRC5,18.0,18.0,1.0,False
25257070,25256330,0,236034852,NaN,Davies Street,tertiary,20 mph,31.843,"LINESTRING (-0.14737 51.51064, -0.14749 51.51092)",no,NaN,NaN,NaN,NaN,FRC5,18.0,18.0,1.0,False
811961032,25257070,0,236034852,NaN,Davies Street,tertiary,20 mph,31.669,"LINESTRING (-0.14731 51.51037, -0.14727 51.510...",no,NaN,NaN,NaN,NaN,FRC5,18.0,18.0,1.0,False
7049006680,25256615,0,202195797,NaN,Davies Street,residential,20 mph,29.996,"LINESTRING (-0.14858 51.51332, -0.14846 51.51306)",no,NaN,NaN,NaN,NaN,FRC5,18.0,18.0,1.0,False
